In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv
/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip
/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip


In [2]:
train=pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [3]:
#시계열적 느낌이 있게 (단어에 의미가 위치에 따라 다를수있다는것을 인식할수있도록 해야한다)
train['Phrase'].iloc[0]

'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [4]:
test=pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip')
test


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [5]:
all_data=pd.concat([train,test])

all_data['Phrase']=all_data['Phrase'].fillna('')

from tensorflow.keras.preprocessing.text import Tokenizer
tk=Tokenizer()

#등록 fitting/전처리 
tk.fit_on_texts(all_data['Phrase'])

2024-03-15 02:40:30.159165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 02:40:30.159281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 02:40:30.424055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
#tk.word_index
#빈도가 높으면 순번이 높다 
#tk.word_counts

In [7]:
#transform 과정 
text=tk.texts_to_sequences(all_data['Phrase'])
print(text[0])

[2, 315, 3, 16573, 7660, 1, 8313, 9, 53, 8, 47, 13, 1, 3940, 8, 187, 47, 13, 1, 13024, 61, 3, 89, 592, 12156, 19, 622, 3, 89, 2810, 5, 52, 3, 2, 42]


In [8]:
#text 사이즈 일반화 시키기 
from tensorflow.keras.preprocessing.sequence import pad_sequences
#text 를 넣어야한다 이미 전처리가 되어있기 때문에 
all_pad=pad_sequences(text)
all_pad.shape

(222352, 52)

In [9]:
#모델링
train_2=all_pad[:len(train)]
test_2=all_pad[len(train):]
train_2


array([[    0,     0,     0, ...,     3,     2,    42],
       [    0,     0,     0, ...,    13,     1,  3940],
       [    0,     0,     0, ...,     0,     2,   315],
       ...,
       [    0,     0,     0, ...,     0, 11171, 11172],
       [    0,     0,     0, ...,     0,     0, 11171],
       [    0,     0,     0, ...,     0,     0, 11172]], dtype=int32)

In [10]:
len(tk.word_index)

17780

In [13]:
from sklearn.model_selection import train_test_split
#stratify 는 validation set 을 나눌때 정답 클래스의 비율이 유지해야되는데 랜덤으로 돌리면 점수 평가할때 힘들어진다 (정답 클래스의 비율이 유지될수있도록)
x_train,x_valid,y_train,y_valid=train_test_split(train_2,train['Sentiment'],test_size=0.15,random_state=42,stratify=train['Sentiment'])
#x_train 와 x_valid 만약 데이터 10000개면 x_train 는 8500개 1500개 나누면 x_train 에 맞는 인덱스로 y_train 과 y_valid 는 정답값만 가지고 있는 애들 

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

#restore_best_weights => 우리모델이 학습을 하다가 마지막 epoch 에 가중치를 넣어서 가져오는게 아니라 가장 좋았던 epoch 에 가중치를 가져오는것을 도와줌 
es=EarlyStopping(patience=3,restore_best_weights=True)

In [21]:
from tensorflow.keras import *
from tensorflow.keras.layers import *

model=Sequential()
#embedding 에서 단어의 뜻을 알아내는 역할을 하는 층//단어+1 0 부터 시작하기때문에//17781 옆에 적은 숫자는 하나의 단어를 학습할때 몇 차원에 vector 로 학습할것인가 (예를 들어 사과라는 단어의 뜻을 1개만 할건지 더 자세히 할것인지//각각의 색깔, 모양 등등 ) 
#하나의 데이터에 차원넣기 세번째옵션 (하나의 데이터에 몇단어씩 이루어져있는지)
model.add(Embedding(17781,10))
#위에 Embedding 층이 2차원이다. (하나의 문장은 단어로 이루어져있고 그 단어안에 또 다른 차원이 존재하기 때문이다 e.g. 52 x 10 (embedding 2차원))/이미지는 3차원 (가로 x 세로 x 높이)
#Flatten 이 있어야하는 이유는 Dense layer 가 1차원이기 때문이다. 
#LSTM 발전된 모델 사용해야됨 
model.add(LSTM(32))
model.add(Dense(5,activation='softmax'))
#loss 가 바뀐다 image generator 에서 바뀐다 분류가 숫자적인걸로 영향을 받는건지 몰라서 
model.compile(metrics=['acc'], optimizer='adam', loss='sparse_categorical_crossentropy')
#모델의 성능을 평가할때 항상 val_loss 를 봐야한다 (val이 들어간것중)
model.fit(x_train,y_train,validation_data=(x_valid,y_valid),epochs=50,callbacks=[es])


Epoch 1/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - acc: 0.5557 - loss: 1.1272 - val_acc: 0.6463 - val_loss: 0.8653
Epoch 2/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - acc: 0.6730 - loss: 0.7963 - val_acc: 0.6587 - val_loss: 0.8319
Epoch 3/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - acc: 0.6960 - loss: 0.7391 - val_acc: 0.6679 - val_loss: 0.8136
Epoch 4/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - acc: 0.7107 - loss: 0.7032 - val_acc: 0.6746 - val_loss: 0.8099
Epoch 5/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - acc: 0.7259 - loss: 0.6632 - val_acc: 0.6714 - val_loss: 0.8172
Epoch 6/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - acc: 0.7361 - loss: 0.6306 - val_acc: 0.6766 - val_loss: 0.8120
Epoch 7/50
4146/4146 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - acc: 0.7491 - loss: 0.6036 - val_acc: 0.6772 - val_loss: 0.8249


In [22]:
#predict 에서는 무조건확률로 나온다 딥러닝 
result=model.predict(test_2)
result

2072/2072 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


array([[1.3218559e-03, 1.4780937e-01, 6.0312736e-01, 2.4421148e-01,
        3.5299344e-03],
       [1.3218559e-03, 1.4780937e-01, 6.0312736e-01, 2.4421148e-01,
        3.5299344e-03],
       [1.5260503e-03, 1.1430061e-01, 7.5989169e-01, 1.2200496e-01,
        2.2766951e-03],
       ...,
       [2.3887930e-02, 4.8934188e-01, 4.7508767e-01, 1.1390027e-02,
        2.9250118e-04],
       [2.3887930e-02, 4.8934188e-01, 4.7508767e-01, 1.1390027e-02,
        2.9250118e-04],
       [1.2746710e-02, 7.4904680e-01, 2.3697942e-01, 1.2173468e-03,
        9.7049606e-06]], dtype=float32)

In [23]:
sub=pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')
sub


,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
...,...,...
66287,222348,2
66288,222349,2
66289,222350,2
66290,222351,2


In [24]:
sub['Sentiment']=result.argmax(1)
sub

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
...,...,...
66287,222348,1
66288,222349,1
66289,222350,1
66290,222351,1


In [25]:
sub.to_csv('sub789.csv',index=0)